### USAUD FUNDING CUTS SENTIMENT ANALYSIS 

### Introduction

.....(light intro text)
.....(TBD)

### Data 

This is data preparation phase for the project. The dataset used here is compiled from two primary sources: Reddit (via web scraping) and NewsAPI (via API calls). Each contributor collected data independently from these platforms, targeting relevant topics for analysis. Below, we begin by importing the collected datasets, merging them, and performing initial cleaning steps to prepare the data for further exploration and modeling.



#### Data Importation

##### news_data

In [ ]:
import os
import pandas as pd

# Set the path to your news_data folder
folder_path = r'N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\raw\news_data'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Read and display columns for each CSV file
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        reddit_mbegoall_df = pd.read_csv(file_path, nrows=0)  # Read only headers
        print(f"Columns in {file}:")
        print(list(reddit_mbegoall_df.columns))
        print("-" * 50)
    except Exception as e:
        print(f"Error reading {file}: {e}")


Columns in Agatha_news.csv:
['keyword', 'source', 'author', 'title', 'description', 'content', 'publishedAt', 'url']
--------------------------------------------------
Columns in Agatha_news_fulltext.csv:
['keyword', 'source', 'author', 'title', 'publishedAt', 'summary', 'text', 'url']
--------------------------------------------------
Columns in cecilia.newsapi.csv:
['keyword', 'source', 'author', 'title', 'description', 'content', 'url', 'publishedAt', 'urlToImage']
--------------------------------------------------
Columns in leo_newsapi_articles_enriched.csv:
['source', 'author', 'title', 'description', 'content', 'url', 'published_at', 'full_text']
--------------------------------------------------
Columns in Mbego_news_usaid_kenya_fulltext.csv:
['source', 'author', 'title', 'description', 'url', 'publishedAt', 'summary', 'full_text']
--------------------------------------------------
Columns in Mbego_news_usaid_kenya_recent.csv:
['source', 'author', 'title', 'description', 'url',

##### reddit_data

In [ ]:

# Set the path to your news_data folder
folder_path = r'N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\raw\reddit_data'

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Read and display columns for each CSV file
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        reddit_mbegoall_df = pd.read_csv(file_path, nrows=0)  # Read only headers
        print(f"Columns in {file}:")
        print(list(reddit_mbegoall_df.columns))
        print("-" * 50)
    except Exception as e:
        print(f"Error reading {file}: {e}")


Columns in Agatha_reddit.csv:
['title', 'selftext', 'subreddit', 'author', 'created_utc', 'url', 'score', 'num_comments', 'keyword']
--------------------------------------------------
Columns in cecilia.redditsubs.csv:
['subreddit', 'keyword', 'title', 'text', 'date_posted', 'upvotes', 'comments', 'url', 'permalink']
--------------------------------------------------
Columns in cecilia.reddit_nbo_ke_africa.csv:
['subreddit', 'keyword', 'title', 'text', 'date_posted', 'upvotes', 'comments', 'url', 'permalink']
--------------------------------------------------
Columns in leo_reddit_posts.csv:
['subreddit', 'search_term', 'title', 'text', 'created_utc', 'created_date', 'score', 'num_comments', 'permalink', 'url']
--------------------------------------------------
Columns in Mbego_reddit_usaid_kenya.csv:
['title', 'score', 'url', 'created', 'subreddit', 'selftext']
--------------------------------------------------
Columns in Mbego_reddit_usaid_kenya2.csv:
['title', 'score', 'url', 'creat

#### Data Merging 

##### news_data



In [ ]:

# Folder containing all News CSVs
folder_path = r'N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\raw\news_data'

# Final save location
save_path = r"N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\processed\individual datasets"

# All .csv files in the news_data folder
news_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Define the final standardized columns
standard_news_cols = [
    'keyword', 'source', 'author', 'title', 'description', 'content',
    'summary', 'full_text', 'publishedAt', 'url', 'language'
]

# Create empty master DataFrame
merged_news_df = pd.DataFrame(columns=standard_news_cols)

# Loop through each file
for file in news_files:
    file_path = os.path.join(folder_path, file)
    try:
        reddit_mbegoall_df = pd.read_csv(file_path)

        # Drop index column if present
        if 'Unnamed: 0' in reddit_mbegoall_df.columns:
            reddit_mbegoall_df.drop(columns=['Unnamed: 0'], inplace=True)

        # Standardize column names
        reddit_mbegoall_df.rename(columns={
            'published_at': 'publishedAt',
            'published_date': 'publishedAt',
            'text': 'content'
        }, inplace=True)

        # Add missing columns
        for col in standard_news_cols:
            if col not in reddit_mbegoall_df.columns:
                reddit_mbegoall_df[col] = pd.NA

        # Align column order
        reddit_mbegoall_df = reddit_mbegoall_df[standard_news_cols]

        # Add to master DataFrame
        merged_news_df = pd.concat([merged_news_df, reddit_mbegoall_df], ignore_index=True)

        print(f"✅ Merged: {file}")
    except Exception as e:
        print(f" Error processing {file}: {e}")

# Save merged file
output_path = os.path.join(save_path, 'Mbego_all_news_merged.csv')
merged_news_df.to_csv(output_path, index=False)

print(f"\n✅ All News files merged and saved to '{output_path}'")


✅ Merged: Agatha_news.csv
✅ Merged: Agatha_news_fulltext.csv
✅ Merged: cecilia.newsapi.csv
✅ Merged: leo_newsapi_articles_enriched.csv
✅ Merged: Mbego_news_usaid_kenya_fulltext.csv
✅ Merged: Mbego_news_usaid_kenya_recent.csv
✅ Merged: newsapi_usaid_articles.csv
✅ Merged: ruth_news.csv

✅ All News files merged and saved to 'N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\processed\individual datasets\Mbego_all_news_merged.csv'


In [42]:

news_merged_path = r"N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\processed\individual datasets\Mbego_all_news_merged.csv"

news_mbegoall_df = pd.read_csv(news_merged_path)

news_mbegoall_df.head(3)


,keyword,source,author,title,description,content,summary,full_text,publishedAt,url,language
0,usaid kenya,Al Jazeera English,Al Jazeera,Has DOGE really saved the US government $180bn?,Elon Musk first claimed the department would m...,President Donald Trump and adviser Elon Musk c...,NaN,NaN,2025-06-06T11:21:51Z,https://www.aljazeera.com/news/2025/6/6/has-do...,NaN
1,usaid kenya,CleanTechnica,Guest Contributor,The Life Story of Ecomobilus Technologies Limi...,By Prof Geoffrey Gitau Here is a story showcas...,By Prof Geoffrey Gitau\r\nHere is a story show...,NaN,NaN,2025-05-26T17:13:41Z,https://cleantechnica.com/2025/05/26/the-life-...,NaN
2,usaid kenya,ProPublica,by Brett Murphy and Anna Maria Barry-Jester,"Death, Sexual Violence and Human Trafficking: ...",by Brett Murphy and Anna Maria Barry-Jester \n...,ProPublica is a nonprofit newsroom that invest...,NaN,NaN,2025-05-28T18:45:00Z,https://www.propublica.org/article/trump-usaid...,NaN


##### reddit_data

In [ ]:

# Define paths
folder_path = r'N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\raw\reddit_data'
save_path = r'N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\processed\individual datasets'

# Define standard columns
standard_cols = [
    'title', 'selftext', 'subreddit', 'author', 'created_utc',
    'created_date', 'score', 'num_comments', 'keyword', 'search_term',
    'date_posted', 'upvotes', 'comments', 'url', 'permalink'
]

# Initialize master DataFrame
merged_df = pd.DataFrame(columns=standard_cols)

# Get all CSV files in folder
reddit_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Loop through each CSV file
for file in reddit_files:
    file_path = os.path.join(folder_path, file)
    try:
        reddit_mbegoall_df = pd.read_csv(file_path)

        # Drop any unnamed index column
        if 'Unnamed: 0' in reddit_mbegoall_df.columns:
            reddit_mbegoall_df.drop(columns=['Unnamed: 0'], inplace=True)

        # Rename common variations
        reddit_mbegoall_df.rename(columns={
            'text': 'selftext',
            'created': 'created_utc'
        }, inplace=True)

        # Add missing columns as empty (NA)
        for col in standard_cols:
            if col not in reddit_mbegoall_df.columns:
                reddit_mbegoall_df[col] = pd.NA

        # Reorder columns to match standard
        reddit_mbegoall_df = reddit_mbegoall_df[standard_cols]

        # Append to the master DataFrame
        merged_df = pd.concat([merged_df, reddit_mbegoall_df], ignore_index=True)

        print(f"✅ Merged: {file}")
    except Exception as e:
        print(f" Error processing {file}: {e}")

# Save merged result
output_file = os.path.join(save_path, 'mbego_all_reddit_merged.csv')
merged_df.to_csv(output_file, index=False)
print(f"\n✅ All Reddit files merged and saved to '{output_file}'")


✅ Merged: Agatha_reddit.csv
✅ Merged: cecilia.redditsubs.csv
✅ Merged: cecilia.reddit_nbo_ke_africa.csv
✅ Merged: leo_reddit_posts.csv
✅ Merged: Mbego_reddit_usaid_kenya.csv
✅ Merged: Mbego_reddit_usaid_kenya2.csv
✅ Merged: reddit_usaid_kenya.csv
✅ Merged: reddit_usaid_sentiment.csv
✅ Merged: ruth_reddit.csv

✅ All Reddit files merged and saved to 'N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\processed\individual datasets\mbego_all_reddit_merged.csv'


In [43]:

reddit_merged_path = r"N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\processed\individual datasets\mbego_all_reddit_merged.csv"

reddit_mbegoall_df = pd.read_csv(reddit_merged_path)

reddit_mbegoall_df.head(3)

,title,selftext,subreddit,author,created_utc,created_date,score,num_comments,keyword,search_term,date_posted,upvotes,comments,url,permalink
0,"USAID left a month ago, do we have ARVs in Kenya?",Someone on a different group (different websit...,Kenya,muerki,2025-04-15 13:16:53,NaN,3.0,5.0,usaid kenya,NaN,NaN,NaN,NaN,https://www.reddit.com/r/Kenya/comments/1jzrn2...,NaN
1,Classism in r/Kenya and r/nairobi,The classism I'm seeing in both subs is a good...,Kenya,Morio_anzenza,2025-04-07 04:21:12,NaN,169.0,95.0,usaid kenya,NaN,NaN,NaN,NaN,https://www.reddit.com/r/Kenya/comments/1jtcvb...,NaN
2,EX-USAID people!! Let's talk,Are you still in contact with the organisation...,Kenya,vindtar,2025-04-05 19:09:10,NaN,2.0,2.0,usaid kenya,NaN,NaN,NaN,NaN,https://www.reddit.com/r/Kenya/comments/1jsb14...,NaN


---

#### Data Understanding 

##### on News Data

Basic Overview

In [44]:
proccessed_final_path = r"N:\Moringa\afterM\Leo NLP 004 USAID 01.06.2025\USAID-Kenya-Sentiment-Analysis\data\processed\news_data\news_data.csv" 

news_df = pd.read_csv(proccessed_final_path)

In [45]:
print(news_df.shape)             # Rows and columns
print(news_df.dtypes)           # Data types                


(2549, 7)
title             object
description       object
text              object
url               object
keyword           object
published_date    object
source_file       object
dtype: object


Missing Data

In [46]:
missing = news_df.isna().sum().sort_values(ascending=False)
print("Missing values per column:\n", missing)

Missing values per column:
 keyword           170
published_date     99
text               25
description        16
url                 2
title               0
source_file         0
dtype: int64


Unique Values per Key Column

In [47]:
print("Unique sources:", news_df['source'].nunique())
print("Unique languages:", news_df['language'].dropna().unique())
print("Sample keywords:", news_df['keyword'].dropna().unique()[:10])

KeyError: 'source'

Date Range

In [10]:
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'], errors='coerce')
print("Date range:", news_df['publishedAt'].min(), "to", news_df['publishedAt'].max())

Date range: 2025-05-09 09:26:01+00:00 to 2025-06-25 17:53:21+00:00


Content Length Check

In [11]:
news_df['content_length'] = news_df['content'].astype(str).apply(len)
print(news_df['content_length'].describe())


count      3200.000000
mean       2314.932813
std        7940.507196
min           3.000000
25%         214.000000
50%         214.000000
75%         215.000000
max      100670.000000
Name: content_length, dtype: float64


Top Sources & Languages

In [12]:
print(news_df['source'].value_counts().head(10))
print(news_df['language'].value_counts().head())

source
Al Jazeera English     206
NPR                    179
Forbes                 154
BBC News               126
ABC News               117
Plos.org               111
Business Insider        89
Daily Signal            58
Time                    53
Yahoo Entertainment     53
Name: count, dtype: int64
Series([], Name: count, dtype: int64)


Duplicates

In [13]:
duplicates = news_df.duplicated(subset=['title', 'url']).sum()
print("Duplicate articles (by title+url):", duplicates)

Duplicate articles (by title+url): 1541


Sample Full Article Text

In [14]:
sample = news_df[['title', 'description', 'content', 'full_text']].dropna().sample(3)
print(sample)

                                                  title  \
2998  Del «fantástico futuro juntos» al «francamente...   
3006  República Democrática del Congo: la fractura p...   
3004  Elon Musk kiosztotta Bonót: „Hazudik vagy ostoba”   

                                            description  \
2998  El propietario de Tesla y X fue el gran apoyo ...   
3006  En la vasta geografía de la República Democrát...   
3004  Bono, a U2 énekese komoly vitát kavart, amikor...   

                                                content  \
2998  ¡Qué lástima, pero adiós! Me despido de ti y m...   
3006  En la vasta geografía de la República Democrát...   
3004  A beszélgetés a The Joe Rogan Experience cím m...   

                                              full_text  
2998  Trump y Musk asisten al lanzamiento de un cohe...  
3006  En la vasta geografía de la República Democrát...  
3004  A beszélgetés a The Joe Rogan Experience című ...  


`Columns Importance`

In [15]:
print(news_df.columns.tolist()) 

['keyword', 'source', 'author', 'title', 'description', 'content', 'summary', 'full_text', 'publishedAt', 'url', 'language', 'content_length']


In [16]:
#column_name:   brief info'

# title:        Useful for headline analysis, summarization, keyword extraction, or sentiment approximation
# description:  concise summary of the article
# content:      Main body of the article. Crucial for any text-based NLP
# source:       Helps identify bias or clustering by publisher; useful in framing analysis
# language:     language filtering
# keyword:      metadata for filtering or guiding classification topics
#*publishedAt:  Useful for temporal analysis, trend detection, or filtering by date (keeping at now if incase we tailor some visualizations as well)

important_cols = ['title', 'description', 'content', 'publishedAt', 'source', 'language', 'keyword']
news_df = news_df[important_cols] 


`Data cleaning (minor_ for quick cleaning)`

- Since title is a `Key` column Removing duplicates from it is neccesary so as to limit redundancy ad will do based on content as well

In [17]:
news_df.drop_duplicates(subset=['title', 'content'], inplace=True)

- Clean language column  

In [18]:
news_df['language'] = news_df['language'].astype(str).str.strip().str.lower()

In [19]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

# Function to detect language from available text
def detect_language(row):
    try:
        # Use content first, fallback to description or title
        text = str(row['content']) or str(row['description']) or str(row['title'])
        return detect(text)
    except LangDetectException:
        return 'unknown'

# Apply to each row
news_df['language'] = news_df.apply(detect_language, axis=1)


In [20]:
print("Unique languages:", news_df['language'].dropna().unique())

Unique languages: ['en' 'tl' 'de' 'pt' 'es' 'no' 'hu' 'tr' 'nl' 'fr' 'ro']


In [21]:
# chose to work with English-only content to ensure accurate analysis and easy interpretation, since it's the only language we fully understand.

news_df = news_df[news_df['language'].isin(['en'])]


In [22]:
#assign my data to avoid re_runs
temp_clean_newsdata = news_df
temp_clean_newsdata.head(4)

,title,description,content,publishedAt,source,language,keyword
0,Has DOGE really saved the US government $180bn?,Elon Musk first claimed the department would m...,President Donald Trump and adviser Elon Musk c...,2025-06-06 11:21:51+00:00,Al Jazeera English,en,usaid kenya
1,The Life Story of Ecomobilus Technologies Limi...,By Prof Geoffrey Gitau Here is a story showcas...,By Prof Geoffrey Gitau\r\nHere is a story show...,2025-05-26 17:13:41+00:00,CleanTechnica,en,usaid kenya
2,"Death, Sexual Violence and Human Trafficking: ...",by Brett Murphy and Anna Maria Barry-Jester \n...,ProPublica is a nonprofit newsroom that invest...,2025-05-28 18:45:00+00:00,ProPublica,en,usaid kenya
3,Congress Should Quickly Approve Trump’s Rescis...,President Donald Trump‘s rescission legislatio...,President Donald Trumps rescission legislation...,2025-06-10 12:00:00+00:00,Daily Signal,en,usaid kenya


In [23]:
temp_clean_newsdata.shape

(1810, 7)

##### on Reddit Data

In [ ]:
print(reddit_mbegoall_df.columns.tolist()) 

['title', 'selftext', 'subreddit', 'author', 'created_utc', 'created_date', 'score', 'num_comments', 'keyword', 'search_term', 'date_posted', 'upvotes', 'comments', 'url', 'permalink']


Drop unneeded columns for modeling

In [ ]:

columns_to_drop = ['created_utc', 'date_posted', 'upvotes', 'url', 'permalink', 'author','created_date']
reddit_mbegoall_df = reddit_mbegoall_df.drop(columns=columns_to_drop, errors='ignore')  # errors='ignore' ensures it won’t crash if a column is missing

#remaining columns
print("Remaining columns:")
print(reddit_mbegoall_df.columns)


Remaining columns:
Index(['title', 'selftext', 'subreddit', 'score', 'num_comments', 'keyword',
       'search_term', 'comments'],
      dtype='object')


Null values check to determine how handle for NLP

In [ ]:
reddit_mbegoall_df.isnull().sum().sort_values(ascending=False)

search_term     1156
comments        1030
keyword          564
num_comments     473
selftext         398
score            276
title              0
subreddit          0
dtype: int64

Cleaning Code for null values 

In [ ]:
# Drop rows where 'created_date' is missing
#reddit_df = reddit_df.dropna(subset=['created_date'])

# Drop 'search_term' if not needed
reddit_mbegoall_df = reddit_mbegoall_df.drop(columns=['search_term'], errors='ignore')

# Fill missing text fields with empty strings
reddit_mbegoall_df['selftext'] = reddit_mbegoall_df['selftext'].fillna('')
reddit_mbegoall_df['comments'] = reddit_mbegoall_df['comments'].fillna('')

# Fill numeric fields with 0
reddit_mbegoall_df['score'] = reddit_mbegoall_df['score'].fillna(0)
reddit_mbegoall_df['num_comments'] = reddit_mbegoall_df['num_comments'].fillna(0)

# Fill missing 'keyword' with 'subreddit' before dropping subreddit
reddit_mbegoall_df['keyword'] = reddit_mbegoall_df['keyword'].fillna(reddit_mbegoall_df['subreddit'])

# Normalize 'keyword' by stripping whitespace and lowercasing
reddit_mbegoall_df['keyword'] = reddit_mbegoall_df['keyword'].str.strip().str.lower()

# Drop 'subreddit' since it's now redundant
reddit_mbegoall_df = reddit_mbegoall_df.drop(columns=['subreddit'], errors='ignore')

# Create unified text field for NLP modeling
reddit_mbegoall_df['full_text'] = reddit_mbegoall_df['title'] + ' ' + reddit_mbegoall_df['selftext']


In [ ]:
reddit_mbegoall_df.nunique().sort_values()


keyword          31
comments         77
num_comments    112
score           238
selftext        544
title           828
full_text       833
dtype: int64

In [ ]:
reddit_mbegoall_df.tail(5)

,title,selftext,score,num_comments,keyword,comments,full_text
1301,Weekly Sub-Saharan Africa Security Situation a...,#Somalia 🇸🇴\r\n#Sudan 🇸🇩\r\nDemocratic Republi...,3.0,2.0,africa,,Weekly Sub-Saharan Africa Security Situation a...
1302,No evidence that Burkina Faso paid off all its...,,52.0,25.0,africa,,No evidence that Burkina Faso paid off all its...
1303,Ghana orders foreigners to exit gold market by...,Ghana has ordered foreigners to exit its gold ...,101.0,12.0,africa,,Ghana orders foreigners to exit gold market by...
1304,Unending Frustration Regarding Sudan War.,https://www.reuters.com/world/britain-boosts-a...,11.0,8.0,africa,,Unending Frustration Regarding Sudan War. http...
1305,Tanzania's Authoritarian Government Has Just B...,Tanzania's main opposition party has been barr...,52.0,14.0,africa,,Tanzania's Authoritarian Government Has Just B...


In [ ]:
reddit_mbegoall_df.shape

(1306, 7)

Top 20 most frequent keywords (after normalization)

In [ ]:
reddit_mbegoall_df['keyword'].value_counts().head(20)


keyword
kenya                                   342
foreign aid, foreign aid                157
worldnews                               138
usaid kenya funding cut                 129
kenya foreign aid                       105
development aid kenya                    99
africa                                   81
usaid                                    66
kenya donor funding                      56
usaid budget cut                         27
usaid, foreign aid, foreign aid          24
usaid suspended funding                  22
usaid funding                            18
usaid kenya                              10
aid withdrawal                            3
usaid, usaid money, donors, ngos          3
foreign aid, foreign aid, trump cuts      3
usaid, usaid money                        3
internationaldev                          3
usaid, ngos                               2
Name: count, dtype: int64

In [32]:
print(news_df.columns.tolist()) 

['title', 'description', 'content', 'publishedAt', 'source', 'language', 'keyword']
